In [1]:
import os
import polars as pl
import import_ipynb
import rfa_utils.general_fns as gf
import rfa_utils.owlracle_api as oa
import rfa_utils.clean_crypto as cc
import rfa_utils.coingecko_api as ca

importing Jupyter notebook from c:\Users\nguye\Documents\Sonderium\Projects\Remittance-Fees-Analysis\rfa_utils\general_fns.ipynb
importing Jupyter notebook from c:\Users\nguye\Documents\Sonderium\Projects\Remittance-Fees-Analysis\rfa_utils\owlracle_api.ipynb
importing Jupyter notebook from c:\Users\nguye\Documents\Sonderium\Projects\Remittance-Fees-Analysis\rfa_utils\clean_crypto.ipynb


ModuleNotFoundError: No module named 'coingecko_api'

In [2]:
# List of stablecoin symbols and IDs
syms = ['usdt', 'usdc', 'busd', 'dai', 'tusd', 'usdp', 'gusd']
sids = ['tether', 'usd-coin', 'binance-usd', 'dai', 'true-usd', 'paxos-standard', 'gemini-dollar']

# Variables for API call
currency = 'usd'
start_date = '2019/01/01'
end_date = '2022/12/31'

In [3]:
# Check and create API data
api_path = os.path.join("data/stablecoin/", "api-data.xlsx")
gf.check_and_create(api_path, ca.create_api_dict, syms, sids, currency, start_date, end_date)

In [4]:
# Check and create fees data
fees_path = os.path.join("data/stablecoin/", "fees-data.xlsx")
gf.check_and_create(fees_path, oa.create_df, start_date, end_date)

In [5]:
# Check if file already exists
combined_path = os.path.join("data/stablecoin/", "combined-data.xlsx")
if not os.path.exists(combined_path):
    # Import Excel files into variables
    api_import = gf.import_excel(
        'data/stablecoin/api-data.xlsx',
        syms
    )
    fees_import = gf.import_excel(
        'data/stablecoin/fees-data.xlsx',
        ['Sheet1']
    )
    # Add the fees dataframe to the API data
    api_import['fees'] = fees_import['Sheet1']
    # Create Excel file with combined data
    gf.create_excel(combined_path, api_import)

In [6]:
# Import combined data
syms.append('fees')
combined = gf.import_excel(
    combined_path,
    syms
)

# Impute column missing values with miceforest
imputed = cc.impute_dfs(combined)

# Winsorize fees column
imputed['fees'] = cc.winsorize_df(imputed['fees'], (0.01, 0.01))